In [1]:
import pandas as pd
import numpy as np


import matplotlib.pyplot as plt
import seaborn as sns

#text processing
import nltk #nltk oder spacey
import string
import re #regular expression

import warnings
warnings.filterwarnings("ignore")

In [2]:
import torch
from torch.utils.data import TensorDataset, Dataset, DataLoader, RandomSampler, SequentialSampler

from keras.preprocessing.sequence import pad_sequences

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report


#model(hugging face)
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, AdamW, get_scheduler

2025-01-24 18:09:23.748706: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-24 18:09:23.787564: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-01-24 18:09:23.787595: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-01-24 18:09:23.787617: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-24 18:09:23.795360: I tensorflow/core/platform/cpu_feature_g

In [3]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AdamW
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [4]:
df = pd.read_excel('df_complete_cleansing.xlsx', index_col=0)
df

,ReceiverID,ActionType,NegoOutcome,Content,Content_Length,Sentence_Count,Word_Count_nltk,NegoOutcomeLabel,Sentiment,SentimentCategory
SenderID,,,,,,,,,,
70,69,Offer,FinalAccept,hope well management company agreed building f...,2529,17,470,1,0.213699,Positive
69,70,Counteroffer,FinalAccept,reaching mei apologize slight delay getting ba...,2579,21,483,1,0.165002,Positive
70,69,Counteroffer,FinalAccept,r kind response need hurry guess face daily fi...,2336,15,454,1,0.222533,Positive
69,70,Counteroffer,FinalAccept,swift reply read set proposal discussed collea...,1961,13,381,1,0.160333,Positive
70,69,Counteroffer,FinalAccept,proposal effort far seems getting close resolv...,1917,10,376,1,0.122500,Positive
...,...,...,...,...,...,...,...,...,...,...
851,856,Counteroffer,FinalReject,latest offer point completely let u take corpo...,1755,21,358,0,0.285340,Positive
856,851,Counteroffer,FinalReject,much latest offer happy could already reached ...,1460,17,294,0,0.282807,Positive
851,856,Counteroffer,FinalReject,meyer fast answer happy found solution issue n...,673,10,144,0,0.251786,Positive


In [5]:
df['Content'] = df['Content'].astype(str)

In [6]:
#nego_message_data=df['Content']
#df_nego_message_data = pd.DataFrame(nego_message_data)
#df_nego_message_data

sentences = df['Content'].tolist()
labels = df['NegoOutcomeLabel'].values


In [7]:
#sentences = df_nego_message_data.Content.values
#sentences = ["[CLS]"+ sentence +" [SEP]" for sentence in sentences]

#labels= df.NegoOutcomeLabel.values

In [8]:
labels

array([1, 1, 1, ..., 0, 0, 0])

In [9]:
#tokenizer=DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
#tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
#print(tokenized_texts[0])
#print(tokenized_texts[1])
model_name = "ProsusAI/finbert"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2, ignore_mismatched_sizes=True)




Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ProsusAI/finbert and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([3, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([3]) in the checkpoint and torch.Size([2]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
tokenized_texts = tokenizer(
    sentences, max_length=512, truncation=True, padding=True, return_tensors="pt"
)

print(tokenized_texts[0])
print(tokenized_texts[1])

Encoding(num_tokens=512, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])
Encoding(num_tokens=512, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])


In [11]:
#MAX_LEN = 512
#input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

# Extrahiere input_ids und attention_mask
input_ids = tokenized_texts["input_ids"]
attention_masks = tokenized_texts["attention_mask"]

In [12]:
#input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype='long', truncating='post', padding='post')

In [13]:
#attention_masks = []

#for seq in input_ids:
#    seq_mask = [float(i>0) for i in seq]
#    attention_masks.append(seq_mask)

In [14]:
train_inputs, validation_inputs, train_labels, validation_labels= train_test_split(input_ids, labels, random_state=42, test_size=0.3)
train_masks, validation_masks, _, _ = train_test_split(attention_masks, input_ids, random_state=42, test_size=0.3)

In [15]:
train_inputs

tensor([[  101,  2720, 11527,  ...,     0,     0,     0],
        [  101,  2171,  3076,  ...,     0,     0,     0],
        [  101,  4541,  3025,  ...,     0,     0,     0],
        ...,
        [  101,  6854,  2172,  ...,     0,     0,     0],
        [  101,  3100,  3305,  ...,     0,     0,     0],
        [  101,  2215, 12367,  ...,     0,     0,     0]])

In [16]:
validation_inputs

tensor([[  101,  2720, 27916,  ...,     0,     0,     0],
        [  101,  2720,  7537,  ...,     0,     0,     0],
        [  101,  4248,  3433,  ...,     0,     0,     0],
        ...,
        [  101, 13676,  3988,  ...,     0,     0,     0],
        [  101, 27916,  2116,  ...,     0,     0,     0],
        [  101,  2748,  2228,  ...,     0,     0,     0]])

In [17]:
train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)

train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)

In [18]:
#je mehr desto langsamer -> batchsize immer wieder ein bisschen anpassen und ausprobieren
batch_size = 32

train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler=RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler=SequentialSampler(validation_data)
validation_dataloader =DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

In [19]:
# Device Setup: Force GPU if available
#import torch
#from transformers import DistilBertForSequenceClassification, DistilBertTokenizer

# Force CPU usage


#device = torch.device("cuda")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
#print("Using CPU for all computations.")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [20]:
#model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels = 2) 
model.to(device)
model.train()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [21]:
optimizer = AdamW(model.parameters(), lr=5e-5)

In [22]:
def compute_accuracy(model, data_loader, device):

    with torch.no_grad():

        correct_pred, num_examples = 0, 0

        for batch_idx, batch in enumerate(data_loader):
            ### Prepare data
            input_ids, input_mask, labels = [t.to(device) for t in batch]  # Move to GPU
        
            outputs = model(input_ids, attention_mask=input_mask, labels=labels)
            loss, logits = outputs['loss'], outputs['logits']
        
            _, predicted_labels = torch.max(logits, 1)
            num_examples += labels.size(0)
            correct_pred += (predicted_labels == labels).sum()

    return correct_pred.float()/num_examples * 100

In [23]:
import time

start_time = time.time()

NUM_EPOCHS = 10

for epoch in range(NUM_EPOCHS):
    
    model.train()
    
    for batch_idx, batch in enumerate(train_dataloader):
        ### Prepare data
        input_ids, input_mask, labels = [t.to(device) for t in batch]  # Move to GPU
        
        ### Forward
        outputs = model(input_ids, attention_mask=input_mask, labels=labels)
        loss, logits = outputs['loss'], outputs['logits']
    
        ### Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        ### Logging
        if not batch_idx % 250:
            print (f'Epoch: {epoch+1:04d}/{NUM_EPOCHS:04d} | '
                   f'Batch {batch_idx:04d}/{len(train_dataloader):04d} | '
                   f'Loss: {loss:.4f}')
            
    model.eval()

    with torch.set_grad_enabled(False):
        print(f'training accuracy: '
              f'{compute_accuracy(model, train_dataloader, device):.2f}%'
              f'\nvalid accuracy: '
              f'{compute_accuracy(model, validation_dataloader, device):.2f}%')
        
    print(f'Time elapsed: {(time.time() - start_time)/60:.2f} min')
    
print(f'Total Training Time: {(time.time() - start_time)/60:.2f} min')

Epoch: 0001/0010 | Batch 0000/0051 | Loss: 0.7134
training accuracy: 68.50%
valid accuracy: 69.43%
Time elapsed: 1.10 min
Epoch: 0002/0010 | Batch 0000/0051 | Loss: 0.6078
training accuracy: 68.50%
valid accuracy: 69.43%
Time elapsed: 2.20 min
Epoch: 0003/0010 | Batch 0000/0051 | Loss: 0.5571
training accuracy: 80.33%
valid accuracy: 73.29%
Time elapsed: 3.30 min
Epoch: 0004/0010 | Batch 0000/0051 | Loss: 0.4360
training accuracy: 86.03%
valid accuracy: 72.86%
Time elapsed: 4.41 min
Epoch: 0005/0010 | Batch 0000/0051 | Loss: 0.4214
training accuracy: 86.89%
valid accuracy: 73.57%
Time elapsed: 5.52 min
Epoch: 0006/0010 | Batch 0000/0051 | Loss: 0.2107
training accuracy: 95.53%
valid accuracy: 75.71%
Time elapsed: 6.62 min
Epoch: 0007/0010 | Batch 0000/0051 | Loss: 0.0773
training accuracy: 95.40%
valid accuracy: 70.29%
Time elapsed: 7.73 min
Epoch: 0008/0010 | Batch 0000/0051 | Loss: 0.2947
training accuracy: 99.02%
valid accuracy: 74.00%
Time elapsed: 8.84 min
Epoch: 0009/0010 | Batch

In [24]:
from sklearn.metrics import classification_report
from collections import Counter

# Nach dem Training:
model.eval()

all_predictions = []
all_labels = []

with torch.no_grad():
    for batch in validation_dataloader:
        ### Prepare data
        input_ids, input_mask, labels = [t.to(device) for t in batch]
        
        ### Forward
        outputs = model(input_ids, attention_mask=input_mask)
        logits = outputs.logits  # Logits enthalten die rohen Vorhersagen des Modells
        
        ### Predictions und Labels sammeln
        predictions = torch.argmax(logits, dim=1)  # Klasse mit höchster Wahrscheinlichkeit
        all_predictions.extend(predictions.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Count der Vorhersagen
prediction_counts = Counter(all_predictions)
print("\nPrediction Counts:")
for label, count in prediction_counts.items():
    print(f"Class {label}: {count} predictions")

# Classification Report
print("\nClassification Report:")
print(classification_report(all_labels, all_predictions, target_names=["Class 0", "Class 1"]))


Prediction Counts:
Class 1: 580 predictions
Class 0: 120 predictions

Classification Report:
              precision    recall  f1-score   support

     Class 0       0.68      0.38      0.49       214
     Class 1       0.77      0.92      0.84       486

    accuracy                           0.76       700
   macro avg       0.73      0.65      0.67       700
weighted avg       0.75      0.76      0.73       700

